# Tabla de clasificación de defectos — v2 (solo *sigmoid* + *brightness/gamma*)

Este notebook es una versión simplificada del *new methods testing*:

- Se han corregido los caracteres raros (mojibake).
- Se han eliminado **todos** los “new methods” (CLAHE, Retinex, WB, etc.).
- Solo queda **sigmoid** (curva S) como preprocesado “extra”.
- Lo que se explora aquí es el efecto de aplicar **sigmoid + (brightness, gamma)**:
  - **Orden A:** `sigmoid -> brightness/gamma`
  - **Orden B:** `brightness/gamma -> sigmoid`
  - (Opcional) `sigmoid` solo y `brightness/gamma` solo como baseline

> Requisito: tener definida la variable de entorno `ROBOFLOW_API_KEY` para poder hacer inferencia con Roboflow.


In [1]:
from __future__ import annotations

from pathlib import Path
import os
import sys
import re
from typing import Any, Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
from PIL import Image, ImageEnhance

# ----------------
# Project root
# ----------------
def find_project_root(start: Path | None = None, marker_dir: str = "data") -> Path:
    """Busca el root del repo subiendo carpetas hasta encontrar `marker_dir/`."""
    start = Path.cwd() if start is None else Path(start).resolve()
    for p in [start, *start.parents]:
        if (p / marker_dir).exists() and (p / marker_dir).is_dir():
            return p
    raise FileNotFoundError(f"No he encontrado '{marker_dir}/' subiendo desde {start}")

PROJECT_ROOT = find_project_root()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

# Imports del repo (para cut opcional)
try:
    from src.raw_image_treatment import potato_pixels_rgb_img
except Exception as e:
    raise ImportError(
        "No he podido importar desde src/raw_image_treatment.py.\n"
        "Asegúrate de ejecutar este notebook dentro del repo y que existe src/raw_image_treatment.py\n"
        f"Error: {e}"
    )

# Roboflow inference SDK
try:
    from inference_sdk import InferenceHTTPClient
except Exception as e:
    raise ImportError(
        "No he podido importar inference_sdk. Instálalo en tu env:\n"
        "  pip install inference-sdk\n"
        f"Error: {e}"
    )


In [2]:
# ------------------------------------------------------------
# Rutas de imágenes (ajusta si tu repo usa otras)
# ------------------------------------------------------------
DIR_DEFINITIVE = PROJECT_ROOT / "data/input/raw/raw_images/definitive"
DIR_TEST = PROJECT_ROOT / "data/input/raw/raw_images/test_1"
DIR_CROPPED_DEF = PROJECT_ROOT / "data/input/processed/cropped_def"

def natural_sort_key(p: Path):
    parts = re.split(r"(\d+)", p.name)
    key: List[Any] = []
    for part in parts:
        key.append(int(part) if part.isdigit() else part.lower())
    return key

def list_images(source: str, pattern: str = "*.png") -> List[Path]:
    if source == "definitive":
        base = DIR_DEFINITIVE
    elif source == "cropped_def":
        base = DIR_CROPPED_DEF
    elif source == "test_1":
        base = DIR_TEST
    else:
        raise ValueError(f"source no soportado: {source}")

    paths = sorted(list(base.glob(pattern)), key=natural_sort_key)
    return paths

def infer_lot_from_name(name: str) -> Optional[int]:
    """Extrae el lote desde nombres tipo p3_..., p4_..., p5_..., p6_..."""
    m = re.search(r"(?:^|_)p(\d+)_", name)
    if m:
        return int(m.group(1))
    m = re.search(r"^p(\d+)_", name)
    if m:
        return int(m.group(1))
    return None

print("definitive exists:", DIR_DEFINITIVE.exists(), "|", DIR_DEFINITIVE)
print("cropped_def exists:", DIR_CROPPED_DEF.exists(), "|", DIR_CROPPED_DEF)
print("test_1 exists:", DIR_TEST.exists(), "|", DIR_TEST)


definitive exists: True | c:\Users\david\Desktop\Uni\potato-dry-matter-optics-ml\data\input\raw\raw_images\definitive
cropped_def exists: True | c:\Users\david\Desktop\Uni\potato-dry-matter-optics-ml\data\input\processed\cropped_def
test_1 exists: True | c:\Users\david\Desktop\Uni\potato-dry-matter-optics-ml\data\input\raw\raw_images\test_1


In [3]:
# ------------------------------------------------------------
# Preprocesado: brightness/gamma + sigmoid (curva S)
# ------------------------------------------------------------
def to_pil_rgb(img_in: Any) -> Image.Image:
    if isinstance(img_in, Image.Image):
        return img_in.convert("RGB")
    if isinstance(img_in, (str, Path, os.PathLike)):
        return Image.open(str(img_in)).convert("RGB")
    if isinstance(img_in, np.ndarray):
        arr = img_in
        if arr.ndim == 2:
            arr = np.stack([arr]*3, axis=-1)
        if arr.shape[-1] == 4:
            arr = arr[..., :3]
        if arr.dtype != np.uint8:
            arr = np.clip(arr, 0, 255).astype(np.uint8)
        return Image.fromarray(arr, mode="RGB")
    raise TypeError(f"Tipo no soportado: {type(img_in)}")

def apply_brightness_and_gamma(pil: Image.Image, brightness: float = 1.0, gamma: float = 1.0) -> Image.Image:
    """Aplica primero brightness (multiplicativo) y luego gamma (curva)."""
    img = pil.convert("RGB")
    if brightness != 1.0:
        img = ImageEnhance.Brightness(img).enhance(float(brightness))

    if gamma != 1.0:
        arr = np.asarray(img).astype(np.float32) / 255.0
        arr = np.power(np.clip(arr, 0.0, 1.0), float(gamma))
        img = Image.fromarray((arr * 255.0 + 0.5).astype(np.uint8), mode="RGB")

    return img

def sigmoid_tone_curve(pil: Image.Image, k: float = 6.0, mid: float = 0.5) -> Image.Image:
    """Curva S (sigmoid) aplicada a la luminancia, preservando crominancia aproximada.

    Implementación sin OpenCV:
    - Calcula luminancia Y en [0,1]
    - Aplica sigmoid: y = 1 / (1 + exp(-k*(Y-mid)))
    - Renormaliza y a [0,1] como en la versión anterior
    - Reescala RGB por y/Y (clamp)
    """
    img = pil.convert("RGB")
    arr = np.asarray(img).astype(np.float32) / 255.0

    # luminancia (Rec.709)
    Y = 0.2126 * arr[..., 0] + 0.7152 * arr[..., 1] + 0.0722 * arr[..., 2]
    y = 1.0 / (1.0 + np.exp(-float(k) * (Y - float(mid))))

    # renormaliza a [0,1] por estabilidad (igual que antes)
    y_min = float(np.min(y))
    y_max = float(np.max(y))
    y = (y - y_min) / (y_max - y_min + 1e-8)

    # reescala por canal para no “lavar” el color
    scale = (y / (Y + 1e-6))[..., None]
    arr2 = np.clip(arr * scale, 0.0, 1.0)

    return Image.fromarray((arr2 * 255.0 + 0.5).astype(np.uint8), mode="RGB")

ORDER_OPTIONS = {
    "sigmoid_only",
    "bg_only",
    "sigmoid_then_bg",
    "bg_then_sigmoid",
}

def apply_preprocess(
    pil: Image.Image,
    *,
    order: str,
    brightness: float,
    gamma: float,
    sigmoid_k: float,
    sigmoid_mid: float,
) -> Image.Image:
    if order not in ORDER_OPTIONS:
        raise ValueError(f"order no soportado: {order}. Opciones: {sorted(ORDER_OPTIONS)}")

    img = pil.convert("RGB")

    if order == "sigmoid_only":
        return sigmoid_tone_curve(img, k=sigmoid_k, mid=sigmoid_mid)

    if order == "bg_only":
        return apply_brightness_and_gamma(img, brightness=brightness, gamma=gamma)

    if order == "sigmoid_then_bg":
        img = sigmoid_tone_curve(img, k=sigmoid_k, mid=sigmoid_mid)
        img = apply_brightness_and_gamma(img, brightness=brightness, gamma=gamma)
        return img

    if order == "bg_then_sigmoid":
        img = apply_brightness_and_gamma(img, brightness=brightness, gamma=gamma)
        img = sigmoid_tone_curve(img, k=sigmoid_k, mid=sigmoid_mid)
        return img

    raise RuntimeError("order no contemplado (bug)")


In [4]:
# ------------------------------------------------------------
# Roboflow: inferencia 1 vez por variante y luego thresholding local
# ------------------------------------------------------------
ROBOFLOW_MODEL_ID = "potato-detection-3et6q/11"
ROBOFLOW_API_URL = "https://serverless.roboflow.com"
ROBOFLOW_API_KEY_ENV = "ROBOFLOW_API_KEY"  # debe existir en tu entorno

def roboflow_infer(pil_img: Image.Image) -> Dict[str, Any]:
    api_key = os.environ.get(ROBOFLOW_API_KEY_ENV)
    if not api_key:
        raise RuntimeError(f"Falta la variable de entorno {ROBOFLOW_API_KEY_ENV}")
    client = InferenceHTTPClient(api_url=ROBOFLOW_API_URL, api_key=api_key)
    return client.infer(pil_img, model_id=ROBOFLOW_MODEL_ID)

def pick_best_prediction(result: Dict[str, Any], thr: float) -> Optional[Dict[str, Any]]:
    preds = result.get("predictions", []) if isinstance(result, dict) else []
    if not preds:
        return None
    best = max(preds, key=lambda p: float(p.get("confidence", 0.0)))
    return best if float(best.get("confidence", 0.0)) >= thr else None

def classify_from_result(result: Dict[str, Any], thr: float) -> Tuple[str, float]:
    best = pick_best_prediction(result, thr)
    if best is None:
        return "Unable to classify", 0.0
    return str(best.get("class", "unknown")), float(best.get("confidence", 0.0))


In [5]:
# ------------------------------------------------------------
# Runner: cut opcional + (sigmoid/bg) + inferencia + thresholds
# ------------------------------------------------------------
def run_defect_table_sigmoid_grid(
    *,
    source: str,
    pattern: str = "*.png",
    max_images: int | None = None,
    do_cut: bool = False,
    cut_margin: int = 35,
    cut_min_conf: float = 0.01,
    thresholds: List[float] = [0.20],
    brightness_values: List[float] = [1.0],
    gamma_values: List[float] = [1.0],
    orders: List[str] = ["bg_then_sigmoid", "sigmoid_then_bg"],
    sigmoid_k: float = 6.0,
    sigmoid_mid: float = 0.5,
) -> pd.DataFrame:
    paths = list_images(source, pattern)
    if max_images is not None:
        paths = paths[: int(max_images)]

    rows: List[Dict[str, Any]] = []

    # Cache para evitar reinferir cuando solo cambian los thresholds
    infer_cache: Dict[Tuple[str, str, float, float, float, float], Dict[str, Any]] = {}
    # (image_name, order, brightness, gamma, k, mid) -> result

    for i, p in enumerate(paths, start=1):
        lot = infer_lot_from_name(p.name)
        base_pil = Image.open(str(p)).convert("RGB")

        # cut opcional (si la imagen NO está ya recortada)
        pil_for_pre = base_pil
        if do_cut and source != "cropped_def":
            cropped, _vis = potato_pixels_rgb_img(pil_for_pre, margin=cut_margin, min_conf=cut_min_conf)
            if cropped is None:
                for thr in thresholds:
                    rows.append({
                        "image": p.name,
                        "lot": lot,
                        "source": source,
                        "order": None,
                        "brightness": None,
                        "gamma": None,
                        "sigmoid_k": sigmoid_k,
                        "sigmoid_mid": sigmoid_mid,
                        "thr": float(thr),
                        "pred_class": None,
                        "confidence": 0.0,
                        "status": "no_potato_detected",
                    })
                continue
            pil_for_pre = cropped.convert("RGB")

        for order in orders:
            for b in brightness_values:
                for g in gamma_values:
                    try:
                        proc = apply_preprocess(
                            pil_for_pre,
                            order=order,
                            brightness=float(b),
                            gamma=float(g),
                            sigmoid_k=float(sigmoid_k),
                            sigmoid_mid=float(sigmoid_mid),
                        )

                        cache_key = (p.name, order, float(b), float(g), float(sigmoid_k), float(sigmoid_mid))
                        if cache_key not in infer_cache:
                            infer_cache[cache_key] = roboflow_infer(proc)

                        result = infer_cache[cache_key]

                        for thr in thresholds:
                            cls, conf = classify_from_result(result, float(thr))
                            rows.append({
                                "image": p.name,
                                "lot": lot,
                                "source": source,
                                "order": order,
                                "brightness": float(b),
                                "gamma": float(g),
                                "sigmoid_k": float(sigmoid_k),
                                "sigmoid_mid": float(sigmoid_mid),
                                "thr": float(thr),
                                "pred_class": cls,
                                "confidence": float(conf),
                                "status": "ok" if cls != "Unable to classify" else "no_pred_above_thr",
                            })

                    except Exception as e:
                        for thr in thresholds:
                            rows.append({
                                "image": p.name,
                                "lot": lot,
                                "source": source,
                                "order": order,
                                "brightness": float(b),
                                "gamma": float(g),
                                "sigmoid_k": float(sigmoid_k),
                                "sigmoid_mid": float(sigmoid_mid),
                                "thr": float(thr),
                                "pred_class": None,
                                "confidence": None,
                                "status": "error",
                                "error": repr(e),
                            })

        if i % 10 == 0:
            print(f"Procesadas {i}/{len(paths)}")

    df = pd.DataFrame(rows)

    # limpieza tipos
    if "confidence" in df.columns:
        df["confidence"] = pd.to_numeric(df["confidence"], errors="coerce")

    return df


In [ ]:
# ==============================
# PARÁMETROS (edita aquí)
# ==============================
SOURCE = "definitive"     # 'definitive', 'cropped_def', 'test_1'
PATTERN = "p3_*.png"      # ej: "p3_*.png" o "*.png"
MAX_IMAGES = None         # None para todas

# Cut (solo si SOURCE != 'cropped_def')
DO_CUT = False
CUT_MARGIN = 35
CUT_MIN_CONF = 0.01

# Threshold(s) de confianza (sigmoid thr)
THRESHOLDS = [0.20]

# Grid brightness/gamma (incluye tu caso 1.6 / 0.9)
BRIGHTNESS_VALUES = [1.0, 1.2, 1.4, 1.6, 1.8]
GAMMA_VALUES = [0.80, 0.90, 1.00, 1.10]

# Órdenes a comparar (y baselines)
ORDERS = ["sigmoid_only", "bg_only", "sigmoid_then_bg", "bg_then_sigmoid"]

# Parámetros de la curva sigmoid
SIGMOID_K = 6.0
SIGMOID_MID = 0.5

df = run_defect_table_sigmoid_grid(
    source=SOURCE,
    pattern=PATTERN,
    max_images=MAX_IMAGES,
    do_cut=DO_CUT,
    cut_margin=CUT_MARGIN,
    cut_min_conf=CUT_MIN_CONF,
    thresholds=THRESHOLDS,
    brightness_values=BRIGHTNESS_VALUES,
    gamma_values=GAMMA_VALUES,
    orders=ORDERS,
    sigmoid_k=SIGMOID_K,
    sigmoid_mid=SIGMOID_MID,
)

print("Rows:", len(df))
display(df.head(10))

# Guardado (opcional)
SAVE_DIR = PROJECT_ROOT / "data/output/defect_calibration/defect_table_sigmoid"
SAVE_DIR.mkdir(parents=True, exist_ok=True)
out_csv = SAVE_DIR / f"table_{SOURCE}_{re.sub(r'[^A-Za-z0-9._-]+','_',PATTERN)}.csv"
df.to_csv(out_csv, index=False)
print("Guardado:", out_csv)


'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)


Procesadas 10/30
Procesadas 20/30
Procesadas 30/30
Rows: 2400


,image,lot,source,order,brightness,gamma,sigmoid_k,sigmoid_mid,thr,pred_class,confidence,status,error
0,p3_1.png,3,definitive,sigmoid_only,1.0,0.8,6.0,0.5,0.2,Potato,0.870509,ok,NaN
1,p3_1.png,3,definitive,sigmoid_only,1.0,0.9,6.0,0.5,0.2,Potato,0.870509,ok,NaN
2,p3_1.png,3,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Potato,0.870509,ok,NaN
3,p3_1.png,3,definitive,sigmoid_only,1.0,1.1,6.0,0.5,0.2,Potato,0.870509,ok,NaN
4,p3_1.png,3,definitive,sigmoid_only,1.2,0.8,6.0,0.5,0.2,Potato,0.870509,ok,NaN
5,p3_1.png,3,definitive,sigmoid_only,1.2,0.9,6.0,0.5,0.2,Potato,0.870509,ok,NaN
6,p3_1.png,3,definitive,sigmoid_only,1.2,1.0,6.0,0.5,0.2,Potato,0.870509,ok,NaN
7,p3_1.png,3,definitive,sigmoid_only,1.2,1.1,6.0,0.5,0.2,Potato,0.870509,ok,NaN
8,p3_1.png,3,definitive,sigmoid_only,1.4,0.8,6.0,0.5,0.2,Potato,0.870509,ok,NaN
9,p3_1.png,3,definitive,sigmoid_only,1.4,0.9,6.0,0.5,0.2,Potato,0.870509,ok,NaN


Guardado: c:\Users\david\Desktop\Uni\potato-dry-matter-optics-ml\data\output\defect_table_sigmoid_v2\table_definitive_p3__.png.csv


In [7]:
# ------------------------------------------------------------
# Mapping: pred_class -> defectuoso (ajusta según las clases que veas)
# ------------------------------------------------------------
thr_for_list = float(min(THRESHOLDS))
df_thr = df[df["thr"] == thr_for_list].copy()

classes = (
    df_thr["pred_class"]
    .fillna("<None>")
    .value_counts(dropna=False)
    .to_frame("count")
)
display(classes)

# Ajusta aquí si la clase "buena" no se llama exactamente así.
GOOD_CLASSES = {"Potato"}

def is_defective(pred_class: str | None) -> Optional[bool]:
    if pred_class is None:
        return None
    if pred_class == "Unable to classify":
        return None
    return (pred_class not in GOOD_CLASSES)

df["pred_defective"] = df["pred_class"].apply(is_defective)

# Expected por lote (heurístico según tu regla)
GOOD_LOTS = {3, 4}
DEFECT_LOTS = {5, 6}

def expected_defective(lot: Optional[int]) -> Optional[bool]:
    if lot is None:
        return None
    if lot in GOOD_LOTS:
        return False
    if lot in DEFECT_LOTS:
        return True
    return None

df["expected_defective"] = df["lot"].apply(expected_defective)

display(df.head(10))


,count
pred_class,
Diseased-fungal potato,1153
Potato,771
Damaged potato,356
Unable to classify,115
Sprouted potato,4
<None>,1


,image,lot,source,order,brightness,gamma,sigmoid_k,sigmoid_mid,thr,pred_class,confidence,status,error,pred_defective,expected_defective
0,p3_1.png,3,definitive,sigmoid_only,1.0,0.8,6.0,0.5,0.2,Potato,0.870509,ok,NaN,False,False
1,p3_1.png,3,definitive,sigmoid_only,1.0,0.9,6.0,0.5,0.2,Potato,0.870509,ok,NaN,False,False
2,p3_1.png,3,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Potato,0.870509,ok,NaN,False,False
3,p3_1.png,3,definitive,sigmoid_only,1.0,1.1,6.0,0.5,0.2,Potato,0.870509,ok,NaN,False,False
4,p3_1.png,3,definitive,sigmoid_only,1.2,0.8,6.0,0.5,0.2,Potato,0.870509,ok,NaN,False,False
5,p3_1.png,3,definitive,sigmoid_only,1.2,0.9,6.0,0.5,0.2,Potato,0.870509,ok,NaN,False,False
6,p3_1.png,3,definitive,sigmoid_only,1.2,1.0,6.0,0.5,0.2,Potato,0.870509,ok,NaN,False,False
7,p3_1.png,3,definitive,sigmoid_only,1.2,1.1,6.0,0.5,0.2,Potato,0.870509,ok,NaN,False,False
8,p3_1.png,3,definitive,sigmoid_only,1.4,0.8,6.0,0.5,0.2,Potato,0.870509,ok,NaN,False,False
9,p3_1.png,3,definitive,sigmoid_only,1.4,0.9,6.0,0.5,0.2,Potato,0.870509,ok,NaN,False,False


In [8]:
# ------------------------------------------------------------
# Métricas por configuración y resumen por lote
# ------------------------------------------------------------
def compute_metrics(sub: pd.DataFrame) -> Dict[str, float]:
    s = sub.dropna(subset=["expected_defective", "pred_defective"]).copy()
    if len(s) == 0:
        return {"n": 0}

    y = s["expected_defective"].astype(bool).to_numpy()
    p = s["pred_defective"].astype(bool).to_numpy()

    tp = int(np.sum((p == 1) & (y == 1)))
    tn = int(np.sum((p == 0) & (y == 0)))
    fp = int(np.sum((p == 1) & (y == 0)))
    fn = int(np.sum((p == 0) & (y == 1)))

    acc = (tp + tn) / max(1, (tp + tn + fp + fn))
    tpr = tp / max(1, (tp + fn))  # recall defectuosas
    tnr = tn / max(1, (tn + fp))  # recall buenas

    return {
        "n": int(len(s)),
        "tp": tp, "tn": tn, "fp": fp, "fn": fn,
        "acc": float(acc),
        "tpr_defective_recall": float(tpr),
        "tnr_good_recall": float(tnr),
    }

group_cols = ["order", "brightness", "gamma", "thr", "sigmoid_k", "sigmoid_mid"]
metrics_rows: List[Dict[str, Any]] = []

for keys, sub in df.groupby(group_cols, dropna=False):
    row = dict(zip(group_cols, keys))
    row.update(compute_metrics(sub))
    metrics_rows.append(row)

metrics = pd.DataFrame(metrics_rows).sort_values(["acc", "tpr_defective_recall"], ascending=False)
display(metrics.head(40))

# Resumen por lote: proporción defectuosa predicha
lot_summary = (
    df.dropna(subset=["lot"])
      .groupby(["lot", "order", "brightness", "gamma", "thr"], as_index=False)["pred_defective"]
      .mean()
      .rename(columns={"pred_defective": "pred_defective_rate"})
      .sort_values(["lot", "pred_defective_rate"], ascending=[True, False])
)
display(lot_summary.head(60))


,order,brightness,gamma,thr,sigmoid_k,sigmoid_mid,n,tp,tn,fp,fn,acc,tpr_defective_recall,tnr_good_recall
23,bg_then_sigmoid,1.0,1.1,0.2,6.0,0.5,27,0,24,3,0,0.888889,0.0,0.888889
63,sigmoid_then_bg,1.0,1.1,0.2,6.0,0.5,27,0,23,4,0,0.851852,0.0,0.851852
22,bg_then_sigmoid,1.0,1.0,0.2,6.0,0.5,29,0,22,7,0,0.758621,0.0,0.758621
40,sigmoid_only,1.0,0.8,0.2,6.0,0.5,29,0,22,7,0,0.758621,0.0,0.758621
41,sigmoid_only,1.0,0.9,0.2,6.0,0.5,29,0,22,7,0,0.758621,0.0,0.758621
42,sigmoid_only,1.0,1.0,0.2,6.0,0.5,29,0,22,7,0,0.758621,0.0,0.758621
43,sigmoid_only,1.0,1.1,0.2,6.0,0.5,29,0,22,7,0,0.758621,0.0,0.758621
44,sigmoid_only,1.2,0.8,0.2,6.0,0.5,29,0,22,7,0,0.758621,0.0,0.758621
45,sigmoid_only,1.2,0.9,0.2,6.0,0.5,29,0,22,7,0,0.758621,0.0,0.758621
46,sigmoid_only,1.2,1.0,0.2,6.0,0.5,29,0,22,7,0,0.758621,0.0,0.758621


,lot,order,brightness,gamma,thr,pred_defective_rate
0,3,bg_only,1.0,0.8,0.2,1.0
1,3,bg_only,1.0,0.9,0.2,1.0
2,3,bg_only,1.0,1.0,0.2,1.0
3,3,bg_only,1.0,1.1,0.2,1.0
4,3,bg_only,1.2,0.8,0.2,1.0
5,3,bg_only,1.2,0.9,0.2,1.0
6,3,bg_only,1.2,1.0,0.2,1.0
7,3,bg_only,1.2,1.1,0.2,1.0
8,3,bg_only,1.4,0.8,0.2,1.0
9,3,bg_only,1.4,0.9,0.2,1.0


In [9]:
# ------------------------------------------------------------
# Comparación directa del orden (¿cambia el resultado?)
# ------------------------------------------------------------
# Filtra solo casos donde ambos órdenes existen (mismo b,g,thr)
df_pair = df[df["order"].isin(["sigmoid_then_bg", "bg_then_sigmoid"])].copy()

pivot = (
    df_pair.pivot_table(
        index=["image", "brightness", "gamma", "thr"],
        columns="order",
        values="pred_class",
        aggfunc="first",
    )
    .reset_index()
)

pivot["order_changes_pred_class"] = (
    pivot["sigmoid_then_bg"].astype(str) != pivot["bg_then_sigmoid"].astype(str)
)

changed = pivot[pivot["order_changes_pred_class"] == True].copy()
print("Casos donde el orden cambia la clase predicha:", len(changed))
display(changed.head(30))


Casos donde el orden cambia la clase predicha: 105


order,image,brightness,gamma,thr,bg_then_sigmoid,sigmoid_then_bg,order_changes_pred_class
10,p3_1.png,1.4,1.0,0.2,Damaged potato,Potato,True
18,p3_1.png,1.8,1.0,0.2,Damaged potato,Unable to classify,True
19,p3_1.png,1.8,1.1,0.2,Damaged potato,Potato,True
30,p3_10.png,1.4,1.0,0.2,Damaged potato,Potato,True
33,p3_10.png,1.6,0.9,0.2,Diseased-fungal potato,Damaged potato,True
34,p3_10.png,1.6,1.0,0.2,Damaged potato,Unable to classify,True
36,p3_10.png,1.8,0.8,0.2,Damaged potato,Diseased-fungal potato,True
37,p3_10.png,1.8,0.9,0.2,Damaged potato,Diseased-fungal potato,True
39,p3_10.png,1.8,1.1,0.2,Damaged potato,Potato,True
43,p3_11.png,1.0,1.1,0.2,Potato,Diseased-fungal potato,True


In [10]:
# ------------------------------------------------------------
# (Opcional) Listado por imagen para una configuración concreta
# ------------------------------------------------------------
# Edita estos valores para ver qué le asigna a cada imagen (sin estadísticas)
SHOW_ORDER = "bg_then_sigmoid"
SHOW_B = 1.6
SHOW_G = 0.9
SHOW_THR = 0.20

sub = df[
    (df["order"] == SHOW_ORDER) &
    (df["brightness"] == float(SHOW_B)) &
    (df["gamma"] == float(SHOW_G)) &
    (df["thr"] == float(SHOW_THR))
].copy().sort_values(["lot", "image"])

display(sub[["image", "lot", "order", "brightness", "gamma", "thr", "pred_class", "confidence", "status"]])


,image,lot,order,brightness,gamma,thr,pred_class,confidence,status
73,p3_1.png,3,bg_then_sigmoid,1.6,0.9,0.2,Diseased-fungal potato,0.762348,ok
793,p3_10.png,3,bg_then_sigmoid,1.6,0.9,0.2,Diseased-fungal potato,0.552730,ok
873,p3_11.png,3,bg_then_sigmoid,1.6,0.9,0.2,Diseased-fungal potato,0.742344,ok
953,p3_12.png,3,bg_then_sigmoid,1.6,0.9,0.2,Diseased-fungal potato,0.871133,ok
1033,p3_13.png,3,bg_then_sigmoid,1.6,0.9,0.2,Damaged potato,0.532295,ok
1113,p3_14.png,3,bg_then_sigmoid,1.6,0.9,0.2,Damaged potato,0.539406,ok
1193,p3_15.png,3,bg_then_sigmoid,1.6,0.9,0.2,Diseased-fungal potato,0.792863,ok
1273,p3_16.png,3,bg_then_sigmoid,1.6,0.9,0.2,Damaged potato,0.639005,ok
1353,p3_17.png,3,bg_then_sigmoid,1.6,0.9,0.2,Diseased-fungal potato,0.730867,ok
1433,p3_18.png,3,bg_then_sigmoid,1.6,0.9,0.2,Diseased-fungal potato,0.897811,ok


In [11]:
import pandas as pd

# --- 1) Encuentra automáticamente el dataframe de resultados ---
def _find_results_df():
    # Prioridad a nombres típicos
    for name in ["results_df", "df_results", "df_res", "results", "df"]:
        if name in globals() and isinstance(globals()[name], pd.DataFrame):
            return globals()[name], name

    # Si no, busca cualquier DataFrame "grande" en globals()
    candidates = [(k, v) for k, v in globals().items() if isinstance(v, pd.DataFrame)]
    if not candidates:
        raise ValueError("No he encontrado ningún DataFrame en globals().")

    # Elige el de más filas (suele ser el de resultados)
    k, v = max(candidates, key=lambda kv: len(kv[1]))
    return v, k

df, df_name = _find_results_df()
print(f"Usando DataFrame: {df_name}  |  filas: {len(df)}")

# --- 2) Detecta columnas que identifican la 'combinación' ---
combo_cols_priority = [
    "combo", "combination", "combo_key"
]
param_cols_priority = [
    "order", "pipeline_order",
    "brightness", "bright",
    "gamma",
    "sigmoid_thr", "sigmoid_threshold", "thr",
    "confidence_threshold", "conf_thr",
]

combo_cols = [c for c in combo_cols_priority if c in df.columns]
if combo_cols:
    group_cols = combo_cols[:1]   # si existe "combo", agrupamos por eso
else:
    group_cols = [c for c in param_cols_priority if c in df.columns]
    if not group_cols:
        # Último recurso: agrupar por todas las columnas "de params" que existan
        raise ValueError(
            "No he encontrado columnas de combinación. "
            "Necesito al menos alguna de: combo/order/brightness/gamma/sigmoid_thr..."
        )

# --- 3) Define qué significa 'ha predicho' (predicción válida) ---
# Busca una columna típica de clase/label/predicción; si no existe, cuenta todas las filas.
pred_col_candidates = [
    "predicted_class", "pred_class", "class_pred", "prediction", "pred",
    "label", "pred_label", "assigned_class", "result_class"
]
pred_col = next((c for c in pred_col_candidates if c in df.columns), None)

if pred_col is not None:
    # predicción válida = no nulo y no vacío y no "no_detection"/"none"/"unknown" (case-insensitive)
    s = df[pred_col].astype(str).str.strip()
    valid_pred = (
        df[pred_col].notna() &
        (s != "") &
        (~s.str.lower().isin(["none", "nan", "no_detection", "nodetection", "unknown", "null"]))
    )
else:
    valid_pred = pd.Series([True] * len(df), index=df.index)

# --- 4) Agrega y ordena ---
agg = (
    df.assign(_valid_pred=valid_pred)
      .groupby(group_cols, dropna=False)
      .agg(
          n_pred_valid=("_valid_pred", "sum"),
          n_rows=("__dummy__" if "__dummy__" in df.columns else df.columns[0], "size"),
      )
      .reset_index()
)

# ratio útil por si hay fallos/skip
agg["ratio_valid"] = agg["n_pred_valid"] / agg["n_rows"]

# --- 5) Muestra TOP y BOTTOM ---
N = 10  # cambia si quieres

agg_sorted = agg.sort_values(["n_pred_valid", "ratio_valid", "n_rows"], ascending=False)

print("\n=== TOP combinaciones con MÁS patatas predichas (válidas) ===")
display(agg_sorted.head(N))

print("\n=== BOTTOM combinaciones con MENOS patatas predichas (válidas) ===")
display(agg_sorted.tail(N).sort_values(["n_pred_valid", "ratio_valid", "n_rows"], ascending=True))


Usando DataFrame: df  |  filas: 2400

=== TOP combinaciones con MÁS patatas predichas (válidas) ===


,order,brightness,gamma,thr,n_pred_valid,n_rows,ratio_valid
0,bg_only,1.0,0.8,0.2,30,30,1.0
1,bg_only,1.0,0.9,0.2,30,30,1.0
2,bg_only,1.0,1.0,0.2,30,30,1.0
3,bg_only,1.0,1.1,0.2,30,30,1.0
4,bg_only,1.2,0.8,0.2,30,30,1.0
5,bg_only,1.2,0.9,0.2,30,30,1.0
6,bg_only,1.2,1.0,0.2,30,30,1.0
7,bg_only,1.2,1.1,0.2,30,30,1.0
8,bg_only,1.4,0.8,0.2,30,30,1.0
9,bg_only,1.4,0.9,0.2,30,30,1.0



=== BOTTOM combinaciones con MENOS patatas predichas (válidas) ===


,order,brightness,gamma,thr,n_pred_valid,n_rows,ratio_valid
47,sigmoid_only,1.2,1.1,0.2,29,30,0.966667
71,sigmoid_then_bg,1.4,1.1,0.2,30,30,1.000000
72,sigmoid_then_bg,1.6,0.8,0.2,30,30,1.000000
73,sigmoid_then_bg,1.6,0.9,0.2,30,30,1.000000
74,sigmoid_then_bg,1.6,1.0,0.2,30,30,1.000000
75,sigmoid_then_bg,1.6,1.1,0.2,30,30,1.000000
76,sigmoid_then_bg,1.8,0.8,0.2,30,30,1.000000
77,sigmoid_then_bg,1.8,0.9,0.2,30,30,1.000000
78,sigmoid_then_bg,1.8,1.0,0.2,30,30,1.000000
79,sigmoid_then_bg,1.8,1.1,0.2,30,30,1.000000


In [13]:
import pandas as pd
import re
import numpy as np

# Llegeix la taula
df = pd.read_csv(out_csv)

# Filtra exactament p3_1..p3_30 (accepta variants tipus "p3_12.png", "p3_12_cropped.png", etc.)
def _idx_p3(name):
    m = re.search(r"\bp3_(\d+)\b", str(name))
    return int(m.group(1)) if m else np.nan

df["_p3_idx"] = df["image"].apply(_idx_p3)
df = df[df["_p3_idx"].between(1, 30)].copy()

# Columnes de combinació
group_cols = ["order", "brightness", "gamma", "sigmoid_k", "sigmoid_mid"]

# Assegura numèrics (per evitar que "1.0" i "1" es tractin raro)
for c in ["brightness", "gamma", "sigmoid_k", "sigmoid_mid"]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Mascaretes: "Potato" i (opcional) status ok
is_potato = df["pred_class"].astype(str).str.strip().str.casefold().eq("potato")
is_ok = df["status"].astype(str).str.strip().str.casefold().eq("ok") if "status" in df.columns else True
df["_potato_ok"] = is_potato & is_ok

# Agregació per combinació: quantes imatges úniques (de 30) han sortit Potato
def _summ(g):
    n_total = g["image"].nunique()
    n_potato = g.loc[g["_potato_ok"], "image"].nunique()
    return pd.Series({"n_total_images": n_total, "n_potato_images": n_potato})

agg = df.groupby(group_cols, dropna=False).apply(_summ).reset_index()
agg["ratio_potato"] = agg["n_potato_images"] / agg["n_total_images"]

# Rànquing
agg_sorted = agg.sort_values(["n_potato_images", "ratio_potato"], ascending=False).reset_index(drop=True)

best = agg_sorted.iloc[0]
print("✅ MILLOR combinació (més 'Potato' en p3_1..p3_30):")
print(
    f"order={best['order']}, brightness={best['brightness']}, gamma={best['gamma']}, "
    f"sigmoid_k={best['sigmoid_k']}, sigmoid_mid={best['sigmoid_mid']}  ->  "
    f"{int(best['n_potato_images'])}/{int(best['n_total_images'])} Potato"
)

print("\n🏁 TOP-10 combinacions:")
display(agg_sorted.head(10))

print("\n🐢 BOTTOM-10 combinacions:")
display(agg_sorted.tail(10).sort_values(["n_potato_images", "ratio_potato"], ascending=True))

✅ MILLOR combinació (més 'Potato' en p3_1..p3_30):
order=bg_then_sigmoid, brightness=1.0, gamma=1.1, sigmoid_k=6.0, sigmoid_mid=0.5  ->  24/30 Potato

🏁 TOP-10 combinacions:


DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.


,order,brightness,gamma,sigmoid_k,sigmoid_mid,n_total_images,n_potato_images,ratio_potato
0,bg_then_sigmoid,1.0,1.1,6.0,0.5,30,24,0.800000
1,sigmoid_then_bg,1.0,1.1,6.0,0.5,30,23,0.766667
2,bg_then_sigmoid,1.0,1.0,6.0,0.5,30,22,0.733333
3,sigmoid_only,1.0,0.8,6.0,0.5,30,22,0.733333
4,sigmoid_only,1.0,0.9,6.0,0.5,30,22,0.733333
5,sigmoid_only,1.0,1.0,6.0,0.5,30,22,0.733333
6,sigmoid_only,1.0,1.1,6.0,0.5,30,22,0.733333
7,sigmoid_only,1.2,0.8,6.0,0.5,30,22,0.733333
8,sigmoid_only,1.2,0.9,6.0,0.5,30,22,0.733333
9,sigmoid_only,1.2,1.0,6.0,0.5,30,22,0.733333



🐢 BOTTOM-10 combinacions:


,order,brightness,gamma,sigmoid_k,sigmoid_mid,n_total_images,n_potato_images,ratio_potato
70,bg_only,1.8,1.1,6.0,0.5,30,0,0.0
71,bg_then_sigmoid,1.2,0.8,6.0,0.5,30,0,0.0
72,bg_then_sigmoid,1.4,0.8,6.0,0.5,30,0,0.0
73,sigmoid_then_bg,1.0,0.8,6.0,0.5,30,0,0.0
74,sigmoid_then_bg,1.2,0.8,6.0,0.5,30,0,0.0
75,sigmoid_then_bg,1.4,0.8,6.0,0.5,30,0,0.0
76,sigmoid_then_bg,1.6,0.8,6.0,0.5,30,0,0.0
77,sigmoid_then_bg,1.6,0.9,6.0,0.5,30,0,0.0
78,sigmoid_then_bg,1.8,0.8,6.0,0.5,30,0,0.0
79,sigmoid_then_bg,1.8,0.9,6.0,0.5,30,0,0.0


In [14]:
import pandas as pd
import re
import numpy as np

# --- Carga (si df no existe) ---
try:
    df
except NameError:
    df = pd.read_csv(out_csv)

# --- Filtra p3_1..p3_30 ---
def _idx_p3(name):
    m = re.search(r"\bp3_(\d+)\b", str(name))
    return int(m.group(1)) if m else np.nan

df["_p3_idx"] = df["image"].apply(_idx_p3)
df_p3 = df[df["_p3_idx"].between(1, 30)].copy()

# --- Define columnas de combinación ---
group_cols = ["order", "brightness", "gamma", "sigmoid_k", "sigmoid_mid"]
for c in ["brightness", "gamma", "sigmoid_k", "sigmoid_mid"]:
    if c in df_p3.columns:
        df_p3[c] = pd.to_numeric(df_p3[c], errors="coerce")

# --- Si no existe agg_sorted, constrúyelo aquí ---
if "agg_sorted" not in globals():
    is_potato = df_p3["pred_class"].astype(str).str.strip().str.casefold().eq("potato")
    is_ok = df_p3["status"].astype(str).str.strip().str.casefold().eq("ok") if "status" in df_p3.columns else True
    df_p3["_potato_ok"] = is_potato & is_ok

    def _summ(g):
        n_total = g["image"].nunique()
        n_potato = g.loc[g["_potato_ok"], "image"].nunique()
        return pd.Series({"n_total_images": n_total, "n_potato_images": n_potato})

    agg = df_p3.groupby(group_cols, dropna=False).apply(_summ).reset_index()
    agg["ratio_potato"] = agg["n_potato_images"] / agg["n_total_images"]
    agg_sorted = agg.sort_values(["n_potato_images", "ratio_potato"], ascending=False).reset_index(drop=True)

# --- Selecciona la combinación TOP ---
top = agg_sorted.iloc[0]
top_key = {c: top[c] for c in group_cols}

print("✅ TOP combinación:")
print(top_key)
print(f"Potato: {int(top['n_potato_images'])}/{int(top['n_total_images'])}  (ratio={top['ratio_potato']:.3f})\n")

# --- Filtra df a esa combinación TOP ---
mask = pd.Series(True, index=df_p3.index)
for c in group_cols:
    # compara con tolerancia si es float
    if pd.api.types.is_numeric_dtype(df_p3[c]):
        mask &= df_p3[c].astype(float).sub(float(top_key[c])).abs() < 1e-12
    else:
        mask &= df_p3[c].astype(str) == str(top_key[c])

top_rows = df_p3[mask].copy()

# --- Ordena por p3_idx y muestra columnas relevantes ---
cols_to_show = ["image", "_p3_idx", "pred_class"]
if "confidence" in top_rows.columns: cols_to_show.append("confidence")
if "status" in top_rows.columns: cols_to_show.append("status")
if "error" in top_rows.columns: cols_to_show.append("error")

top_rows = top_rows.sort_values("_p3_idx")[cols_to_show]

# --- Imprime bonito (sin display si prefieres) ---
print("📌 Resultados (p3_1..p3_30) para la TOP combinación:\n")
for _, r in top_rows.iterrows():
    img = r["image"]
    cls = r["pred_class"]
    conf = r["confidence"] if "confidence" in r else None
    status = r["status"] if "status" in r else None
    extra = []
    if conf is not None and pd.notna(conf): extra.append(f"conf={float(conf):.3f}")
    if status is not None and pd.notna(status): extra.append(f"status={status}")
    print(f"{img:25s} -> {cls:12s} " + (" | " + ", ".join(extra) if extra else ""))

# (Opcional) también en tabla
display(top_rows.reset_index(drop=True))


✅ TOP combinación:
{'order': 'bg_then_sigmoid', 'brightness': 1.0, 'gamma': 1.1, 'sigmoid_k': 6.0, 'sigmoid_mid': 0.5}
Potato: 24/30  (ratio=0.800)

📌 Resultados (p3_1..p3_30) para la TOP combinación:

p3_1.png                  -> Potato        | conf=0.842, status=ok
p3_2.png                  -> Potato        | conf=0.812, status=ok
p3_3.png                  -> Potato        | conf=0.772, status=ok
p3_4.png                  -> Potato        | conf=0.545, status=ok
p3_5.png                  -> Potato        | conf=0.468, status=ok
p3_6.png                  -> Potato        | conf=0.865, status=ok
p3_7.png                  -> Potato        | conf=0.831, status=ok
p3_8.png                  -> Potato        | conf=0.502, status=ok
p3_9.png                  -> Potato        | conf=0.790, status=ok
p3_10.png                 -> Potato        | conf=0.851, status=ok
p3_11.png                 -> Potato        | conf=0.467, status=ok
p3_12.png                 -> Potato        | conf=0.740, stat

,image,_p3_idx,pred_class,confidence,status,error
0,p3_1.png,1,Potato,0.841876,ok,NaN
1,p3_2.png,2,Potato,0.812458,ok,NaN
2,p3_3.png,3,Potato,0.772239,ok,NaN
3,p3_4.png,4,Potato,0.544951,ok,NaN
4,p3_5.png,5,Potato,0.468186,ok,NaN
5,p3_6.png,6,Potato,0.865184,ok,NaN
6,p3_7.png,7,Potato,0.830681,ok,NaN
7,p3_8.png,8,Potato,0.501873,ok,NaN
8,p3_9.png,9,Potato,0.789568,ok,NaN
9,p3_10.png,10,Potato,0.851078,ok,NaN


In [16]:
from pathlib import Path
import pandas as pd
import numpy as np
from PIL import Image
import traceback
from src.raw_image_treatment import potato_defect_classification, apply_brightness_and_gamma

# --------- Sanity checks ----------
if "potato_defect_classification" not in globals():
    raise RuntimeError("No trobo 'potato_defect_classification' a globals(). Importa-la primer des de src/raw_image_treatment.")
if "apply_brightness_and_gamma" not in globals():
    raise RuntimeError("No trobo 'apply_brightness_and_gamma' a globals(). Importa-la primer des de src/raw_image_treatment.")
if "top_key" not in globals():
    raise RuntimeError("No trobo 'top_key'. Executa abans la cel·la que calcula la TOP combinació (top_key).")

# --------- Params (agafa defaults del notebook si existeixen) ----------
DEFAULT_K   = float(globals().get("SIGMOID_K", 6.0))
DEFAULT_MID = float(globals().get("SIGMOID_MID", 0.5))
DEFAULT_THR = float(globals().get("SIGMOID_THR", globals().get("THR", 0.2)))
CONF_THRESH = float(globals().get("CONFIDENCE_THRESHOLD", globals().get("CONF_THR", 0.0)))

SOURCE_NAME = "definitive"
LOT = 4

# --------- Sigmoid (si no existeix al notebook) ----------
def apply_sigmoid_pil(img_pil: Image.Image, k: float = 6.0, mid: float = 0.5) -> Image.Image:
    img = img_pil.convert("RGB")
    arr = np.asarray(img).astype(np.float32) / 255.0
    out = 1.0 / (1.0 + np.exp(-k * (arr - mid)))
    out = np.clip(out * 255.0, 0, 255).astype(np.uint8)
    return Image.fromarray(out, mode="RGB")

def preprocess(img_pil: Image.Image, cfg: dict) -> Image.Image:
    order = str(cfg["order"]).strip().lower()
    b = float(cfg["brightness"])
    g = float(cfg["gamma"])
    k = float(cfg["sigmoid_k"])
    mid = float(cfg["sigmoid_mid"])

    img = img_pil.convert("RGB")

    if order in ["sigmoid_only"]:
        return apply_sigmoid_pil(img, k=k, mid=mid)

    if order in ["bg_only"]:
        return apply_brightness_and_gamma(img, brightness=b, gamma=g)

    if order in ["sigmoid_then_bg", "sigmoid->bg", "sigmoid_then_brightness_gamma"]:
        img2 = apply_sigmoid_pil(img, k=k, mid=mid)
        return apply_brightness_and_gamma(img2, brightness=b, gamma=g)

    if order in ["bg_then_sigmoid", "bg->sigmoid", "brightness_gamma_then_sigmoid"]:
        img2 = apply_brightness_and_gamma(img, brightness=b, gamma=g)
        return apply_sigmoid_pil(img2, k=k, mid=mid)

    # fallback: assume sigmoid_then_bg
    img2 = apply_sigmoid_pil(img, k=k, mid=mid)
    return apply_brightness_and_gamma(img2, brightness=b, gamma=g)

# --------- Trobar imatges p4 ----------
# Prova rutes típiques dins el repo
candidate_dirs = []
if "PROJECT_ROOT" in globals():
    PROJECT_ROOT = globals()["PROJECT_ROOT"]
    candidate_dirs += [
        Path(PROJECT_ROOT) / "data/input/raw/raw_images/definitive",
        Path(PROJECT_ROOT) / "data/input/raw/definitive",
        Path(PROJECT_ROOT) / "data/input/raw/raw_images",
        Path(PROJECT_ROOT) / "data/input/raw",
    ]

img_paths = []
for d in candidate_dirs:
    if d.exists():
        img_paths += sorted(d.glob("p4_*.png"))

# Si no ha trobat res, prova al cwd recursiu
if not img_paths:
    img_paths = sorted(Path(".").rglob("p4_*.png"))

if not img_paths:
    raise FileNotFoundError("No he trobat cap imatge 'p4_*.png' al repo. Revisa on tens guardades les p4.")

print(f"Trobades {len(img_paths)} imatges p4_*.png")

# --------- Configs: baseline + TOP ----------
baseline_cfg = {
    "order": "sigmoid_only",
    "brightness": 1.0,
    "gamma": 1.0,
    "sigmoid_k": DEFAULT_K,
    "sigmoid_mid": DEFAULT_MID,
    "thr": DEFAULT_THR,
}

top_cfg = {
    "order": top_key["order"],
    "brightness": float(top_key["brightness"]),
    "gamma": float(top_key["gamma"]),
    "sigmoid_k": float(top_key["sigmoid_k"]),
    "sigmoid_mid": float(top_key["sigmoid_mid"]),
    "thr": DEFAULT_THR,  # si vols, canvia-ho per un thr diferent
}

def run_pipeline(cfg: dict) -> pd.DataFrame:
    rows = []
    for i, p in enumerate(img_paths, start=1):
        try:
            img = Image.open(p).convert("RGB")
            img_proc = preprocess(img, cfg)

            pred = potato_defect_classification(img_proc, confidence_threshold=CONF_THRESH)

            # pred pot ser dict o tuple segons la teva funció -> fem robust
            if isinstance(pred, dict):
                pred_class = pred.get("pred_class") or pred.get("class") or pred.get("prediction")
                confidence = pred.get("confidence") or pred.get("conf") or pred.get("score")
            elif isinstance(pred, (list, tuple)) and len(pred) >= 2:
                pred_class, confidence = pred[0], pred[1]
            else:
                pred_class, confidence = pred, None

            rows.append({
                "image": p.name,
                "lot": LOT,
                "source": SOURCE_NAME,
                "order": cfg["order"],
                "brightness": cfg["brightness"],
                "gamma": cfg["gamma"],
                "sigmoid_k": cfg["sigmoid_k"],
                "sigmoid_mid": cfg["sigmoid_mid"],
                "thr": cfg["thr"],
                "pred_class": pred_class,
                "confidence": confidence,
                "status": "ok",
                "error": "",
            })
        except Exception as e:
            rows.append({
                "image": p.name,
                "lot": LOT,
                "source": SOURCE_NAME,
                "order": cfg["order"],
                "brightness": cfg["brightness"],
                "gamma": cfg["gamma"],
                "sigmoid_k": cfg["sigmoid_k"],
                "sigmoid_mid": cfg["sigmoid_mid"],
                "thr": cfg["thr"],
                "pred_class": "",
                "confidence": "",
                "status": "error",
                "error": str(e),
            })

        if i % 10 == 0:
            print(f"[{i}/{len(img_paths)}] {cfg['order']} ...")

    return pd.DataFrame(rows)

# --------- Executa ----------
print("\n=== RUN baseline (sigmoid normal) ===")
df_p4_sigmoid = run_pipeline(baseline_cfg)

print("\n=== RUN TOP combinació ===")
df_p4_top = run_pipeline(top_cfg)

# --------- Comptes Potato ----------
def count_potato(df_):
    ok = df_["status"].astype(str).str.strip().str.casefold().eq("ok")
    potato = df_["pred_class"].astype(str).str.strip().str.casefold().eq("potato")
    return int((ok & potato).sum())

n_pot_sig = count_potato(df_p4_sigmoid)
n_pot_top = count_potato(df_p4_top)

print("\n📊 #Potato (p4):")
print(f" - Sigmoid normal : {n_pot_sig}/{len(df_p4_sigmoid)}")
print(f" - TOP combinació : {n_pot_top}/{len(df_p4_top)}")

# --------- Guarda CSVs (si tens SAVE_DIR definit, l'usa; sinó crea un per p4) ----------
if "SAVE_DIR" in globals():
    save_dir = Path(SAVE_DIR)
else:
    if "PROJECT_ROOT" in globals():
        save_dir = Path(PROJECT_ROOT) / "data/output/defect_table_sigmoid_v2"
    else:
        save_dir = Path("data/output/defect_table_sigmoid_v2")
    save_dir.mkdir(parents=True, exist_ok=True)

out1 = save_dir / "table_definitive_p4_sigmoid_normal.csv"
out2 = save_dir / "table_definitive_p4_top_combo.csv"
df_p4_sigmoid.to_csv(out1, index=False)
df_p4_top.to_csv(out2, index=False)

print(f"\n✅ Guardat:\n- {out1}\n- {out2}")

# --------- Mostra taules ----------
print("\n=== Taula: p4 sigmoid normal ===")
display(df_p4_sigmoid)

print("\n=== Taula: p4 TOP combinació ===")
display(df_p4_top)


Trobades 72 imatges p4_*.png

=== RUN baseline (sigmoid normal) ===


'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)


[10/72] sigmoid_only ...
[20/72] sigmoid_only ...
[30/72] sigmoid_only ...
[40/72] sigmoid_only ...
[50/72] sigmoid_only ...
[60/72] sigmoid_only ...
[70/72] sigmoid_only ...

=== RUN TOP combinació ===
[10/72] bg_then_sigmoid ...
[20/72] bg_then_sigmoid ...
[30/72] bg_then_sigmoid ...
[40/72] bg_then_sigmoid ...
[50/72] bg_then_sigmoid ...
[60/72] bg_then_sigmoid ...
[70/72] bg_then_sigmoid ...

📊 #Potato (p4):
 - Sigmoid normal : 1/72
 - TOP combinació : 1/72

✅ Guardat:
- c:\Users\david\Desktop\Uni\potato-dry-matter-optics-ml\data\output\defect_table_sigmoid_v2\table_definitive_p4_sigmoid_normal.csv
- c:\Users\david\Desktop\Uni\potato-dry-matter-optics-ml\data\output\defect_table_sigmoid_v2\table_definitive_p4_top_combo.csv

=== Taula: p4 sigmoid normal ===


,image,lot,source,order,brightness,gamma,sigmoid_k,sigmoid_mid,thr,pred_class,confidence,status,error
0,p4_1.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Diseased-fungal potato,0.802144,ok,
1,p4_10.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Diseased-fungal potato,0.802945,ok,
2,p4_11.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Diseased-fungal potato,0.678008,ok,
3,p4_12.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Diseased-fungal potato,0.559069,ok,
4,p4_13.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Diseased-fungal potato,0.492303,ok,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,p4_70.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Damaged potato,0.411038,ok,
68,p4_71.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Damaged potato,0.747263,ok,
69,p4_72.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Diseased-fungal potato,0.822295,ok,
70,p4_8.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Diseased-fungal potato,0.858972,ok,



=== Taula: p4 TOP combinació ===


,image,lot,source,order,brightness,gamma,sigmoid_k,sigmoid_mid,thr,pred_class,confidence,status,error
0,p4_1.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Diseased-fungal potato,0.832088,ok,
1,p4_10.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Diseased-fungal potato,0.753998,ok,
2,p4_11.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Diseased-fungal potato,0.569407,ok,
3,p4_12.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Diseased-fungal potato,0.402411,ok,
4,p4_13.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Diseased-fungal potato,0.474694,ok,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,p4_70.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Unable to classify,0.000000,ok,
68,p4_71.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Damaged potato,0.784907,ok,
69,p4_72.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Diseased-fungal potato,0.762332,ok,
70,p4_8.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Diseased-fungal potato,0.838756,ok,


In [17]:
from pathlib import Path
import pandas as pd
import numpy as np
import re
from PIL import Image

# -----------------------------
# 0) Requisits
# -----------------------------
if "potato_defect_classification" not in globals():
    raise RuntimeError("No trobo potato_defect_classification. Importa-la primer (src.raw_image_treatment).")
if "apply_brightness_and_gamma" not in globals():
    raise RuntimeError("No trobo apply_brightness_and_gamma. Importa-la primer (src.raw_image_treatment).")

# -----------------------------
# 1) Localitza funció SIGMOID del teu notebook (sense reinventar)
# -----------------------------
sigmoid_fn = None

# 1a) Si al notebook ja existeix una funció amb nom "apply_sigmoid" o similar
for cand in ["apply_sigmoid", "sigmoid_transform", "sigmoid_contrast", "apply_sigmoid_contrast"]:
    if cand in globals() and callable(globals()[cand]):
        sigmoid_fn = globals()[cand]
        break

# 1b) Si no, prova importar-la del src (si existeix)
if sigmoid_fn is None:
    try:
        from src.raw_image_treatment import apply_sigmoid as _apply_sigmoid
        sigmoid_fn = _apply_sigmoid
    except Exception:
        sigmoid_fn = None

# 1c) Fallback (només si NO existeix cap sigmoid al teu notebook)
def _fallback_sigmoid(img_pil: Image.Image, k: float = 6.0, mid: float = 0.5) -> Image.Image:
    img = img_pil.convert("RGB")
    arr = np.asarray(img).astype(np.float32) / 255.0
    out = 1.0 / (1.0 + np.exp(-k * (arr - mid)))
    out = np.clip(out * 255.0, 0, 255).astype(np.uint8)
    return Image.fromarray(out, mode="RGB")

if sigmoid_fn is None:
    print("⚠️ No he trobat cap funció sigmoid al notebook/src. Faré servir un fallback estàndard.")
    sigmoid_fn = _fallback_sigmoid
else:
    print(f"✅ Usant sigmoid del notebook/src: {sigmoid_fn.__name__}")

# Wrapper robust per cridar sigmoid_fn amb (img, k, mid) encara que la signatura variï
def apply_sigmoid(img_pil: Image.Image, k: float, mid: float) -> Image.Image:
    try:
        return sigmoid_fn(img_pil, k=k, mid=mid)
    except TypeError:
        try:
            return sigmoid_fn(img_pil, k, mid)
        except TypeError:
            # si la teva funció no admet k/mid, simplement aplica-la
            return sigmoid_fn(img_pil)

# -----------------------------
# 2) Configs exactes
# -----------------------------
TOP_CFG = {
    "order": "bg_then_sigmoid",
    "brightness": 1.0,
    "gamma": 1.1,
    "sigmoid_k": 6.0,
    "sigmoid_mid": 0.5,
}

# "Sigmoid normal del notebook":
# - Si tens un dict ja definit, el respectem
# - Si tens variables sueltes, les agafem
# - Si no, fem default a sigmoid_only amb b=g=1
BASELINE_CFG = None
for name in ["SIGMOID_NORMAL_CFG", "sigmoid_normal_cfg", "BASELINE_SIGMOID_CFG", "DEFAULT_SIGMOID_CFG"]:
    if name in globals() and isinstance(globals()[name], dict):
        BASELINE_CFG = dict(globals()[name])
        break

if BASELINE_CFG is None:
    BASELINE_CFG = {
        "order": globals().get("SIGMOID_NORMAL_ORDER", globals().get("ORDER_BASELINE", "sigmoid_only")),
        "brightness": float(globals().get("SIGMOID_NORMAL_BRIGHTNESS", globals().get("BRIGHTNESS_BASELINE", 1.0))),
        "gamma": float(globals().get("SIGMOID_NORMAL_GAMMA", globals().get("GAMMA_BASELINE", 1.0))),
        "sigmoid_k": float(globals().get("SIGMOID_K", 6.0)),
        "sigmoid_mid": float(globals().get("SIGMOID_MID", 0.5)),
    }

# Thr i confidence (per guardar a la taula; el model usa confidence_threshold)
THR = float(globals().get("THR", globals().get("SIGMOID_THR", 0.2)))
CONF_THRESH = float(globals().get("CONFIDENCE_THRESHOLD", globals().get("CONF_THR", 0.0)))

print("\n=== CONFIGS ===")
print("BASELINE (sigmoid normal):", BASELINE_CFG)
print("TOP:", TOP_CFG)
print(f"thr={THR} | confidence_threshold={CONF_THRESH}\n")

# -----------------------------
# 3) Trobar imatges p4_*.png
# -----------------------------
candidate_dirs = []
if "PROJECT_ROOT" in globals():
    PROJECT_ROOT = Path(globals()["PROJECT_ROOT"])
    candidate_dirs += [
        PROJECT_ROOT / "data/input/raw/raw_images/definitive",
        PROJECT_ROOT / "data/input/raw/definitive",
        PROJECT_ROOT / "data/input/raw/raw_images",
        PROJECT_ROOT / "data/input/raw",
    ]

img_paths = []
for d in candidate_dirs:
    if d.exists():
        img_paths += sorted(d.glob("p4_*.png"))

if not img_paths:
    img_paths = sorted(Path(".").rglob("p4_*.png"))

if not img_paths:
    raise FileNotFoundError("No he trobat cap imatge p4_*.png al repo.")

def _idx_p4(name):
    m = re.search(r"\bp4_(\d+)\b", str(name))
    return int(m.group(1)) if m else 10**9

img_paths = sorted(img_paths, key=lambda p: _idx_p4(p.name))
print(f"Trobades {len(img_paths)} imatges p4_*.png")

# -----------------------------
# 4) Preproc segons order
# -----------------------------
def preprocess(img_pil: Image.Image, cfg: dict) -> Image.Image:
    order = str(cfg["order"]).strip().lower()
    b = float(cfg["brightness"])
    g = float(cfg["gamma"])
    k = float(cfg["sigmoid_k"])
    mid = float(cfg["sigmoid_mid"])

    img = img_pil.convert("RGB")

    if order == "sigmoid_only":
        return apply_sigmoid(img, k=k, mid=mid)

    if order == "bg_only":
        return apply_brightness_and_gamma(img, brightness=b, gamma=g)

    if order == "sigmoid_then_bg":
        img2 = apply_sigmoid(img, k=k, mid=mid)
        return apply_brightness_and_gamma(img2, brightness=b, gamma=g)

    if order == "bg_then_sigmoid":
        img2 = apply_brightness_and_gamma(img, brightness=b, gamma=g)
        return apply_sigmoid(img2, k=k, mid=mid)

    # fallback segur
    img2 = apply_brightness_and_gamma(img, brightness=b, gamma=g)
    return apply_sigmoid(img2, k=k, mid=mid)

# -----------------------------
# 5) Execució per p4 i taules
# -----------------------------
def run_cfg_on_p4(cfg: dict, lot: int = 4, source: str = "definitive") -> pd.DataFrame:
    rows = []
    for p in img_paths:
        try:
            img = Image.open(p).convert("RGB")
            img_proc = preprocess(img, cfg)

            pred = potato_defect_classification(img_proc, confidence_threshold=CONF_THRESH)

            # parsing robust del retorn
            pred_class, confidence = None, None
            if isinstance(pred, dict):
                pred_class = pred.get("pred_class") or pred.get("class") or pred.get("prediction")
                confidence = pred.get("confidence") or pred.get("conf") or pred.get("score")
            elif isinstance(pred, (list, tuple)) and len(pred) >= 2:
                pred_class, confidence = pred[0], pred[1]
            else:
                pred_class = pred

            rows.append({
                "image": p.name,
                "lot": lot,
                "source": source,
                "order": cfg["order"],
                "brightness": cfg["brightness"],
                "gamma": cfg["gamma"],
                "sigmoid_k": cfg["sigmoid_k"],
                "sigmoid_mid": cfg["sigmoid_mid"],
                "thr": THR,
                "pred_class": pred_class,
                "confidence": confidence,
                "status": "ok",
                "error": "",
            })
        except Exception as e:
            rows.append({
                "image": p.name,
                "lot": lot,
                "source": source,
                "order": cfg["order"],
                "brightness": cfg["brightness"],
                "gamma": cfg["gamma"],
                "sigmoid_k": cfg["sigmoid_k"],
                "sigmoid_mid": cfg["sigmoid_mid"],
                "thr": THR,
                "pred_class": "",
                "confidence": "",
                "status": "error",
                "error": str(e),
            })

    return pd.DataFrame(rows)

def count_potato(df_):
    ok = df_["status"].astype(str).str.strip().str.casefold().eq("ok")
    potato = df_["pred_class"].astype(str).str.strip().str.casefold().eq("potato")
    return int((ok & potato).sum())

df_p4_sigmoid_normal = run_cfg_on_p4(BASELINE_CFG)
df_p4_top_combo = run_cfg_on_p4(TOP_CFG)

n_sig = count_potato(df_p4_sigmoid_normal)
n_top = count_potato(df_p4_top_combo)

print("\n📊 #Potato en p4:")
print(f" - Sigmoid normal : {n_sig}/{len(df_p4_sigmoid_normal)}")
print(f" - TOP combinació : {n_top}/{len(df_p4_top_combo)}")

# Guardar (si tens SAVE_DIR el respecta)
if "SAVE_DIR" in globals():
    save_dir = Path(SAVE_DIR)
else:
    save_dir = Path("data/output/defect_table_sigmoid_v2")
    save_dir.mkdir(parents=True, exist_ok=True)

out_base = save_dir / "table_definitive_p4_sigmoid_normal.csv"
out_top  = save_dir / "table_definitive_p4_top_combo.csv"
df_p4_sigmoid_normal.to_csv(out_base, index=False)
df_p4_top_combo.to_csv(out_top, index=False)

print(f"\n✅ CSV guardats:\n- {out_base}\n- {out_top}\n")

print("=== Taula: p4 sigmoid normal ===")
display(df_p4_sigmoid_normal)

print("=== Taula: p4 TOP combinació ===")
display(df_p4_top_combo)


⚠️ No he trobat cap funció sigmoid al notebook/src. Faré servir un fallback estàndard.

=== CONFIGS ===
BASELINE (sigmoid normal): {'order': 'sigmoid_only', 'brightness': 1.0, 'gamma': 1.0, 'sigmoid_k': 6.0, 'sigmoid_mid': 0.5}
TOP: {'order': 'bg_then_sigmoid', 'brightness': 1.0, 'gamma': 1.1, 'sigmoid_k': 6.0, 'sigmoid_mid': 0.5}
thr=0.2 | confidence_threshold=0.0

Trobades 72 imatges p4_*.png


'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)



📊 #Potato en p4:
 - Sigmoid normal : 1/72
 - TOP combinació : 1/72

✅ CSV guardats:
- c:\Users\david\Desktop\Uni\potato-dry-matter-optics-ml\data\output\defect_table_sigmoid_v2\table_definitive_p4_sigmoid_normal.csv
- c:\Users\david\Desktop\Uni\potato-dry-matter-optics-ml\data\output\defect_table_sigmoid_v2\table_definitive_p4_top_combo.csv

=== Taula: p4 sigmoid normal ===


,image,lot,source,order,brightness,gamma,sigmoid_k,sigmoid_mid,thr,pred_class,confidence,status,error
0,p4_1.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Diseased-fungal potato,0.802043,ok,
1,p4_2.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Diseased-fungal potato,0.847370,ok,
2,p4_3.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Diseased-fungal potato,0.847291,ok,
3,p4_4.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Diseased-fungal potato,0.752796,ok,
4,p4_5.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Diseased-fungal potato,0.430456,ok,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,p4_68.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Diseased-fungal potato,0.807448,ok,
68,p4_69.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Damaged potato,0.546293,ok,
69,p4_70.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Damaged potato,0.411721,ok,
70,p4_71.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Damaged potato,0.746927,ok,


=== Taula: p4 TOP combinació ===


,image,lot,source,order,brightness,gamma,sigmoid_k,sigmoid_mid,thr,pred_class,confidence,status,error
0,p4_1.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Diseased-fungal potato,0.832088,ok,
1,p4_2.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Diseased-fungal potato,0.829076,ok,
2,p4_3.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Diseased-fungal potato,0.839306,ok,
3,p4_4.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Diseased-fungal potato,0.675565,ok,
4,p4_5.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Diseased-fungal potato,0.463931,ok,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,p4_68.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Diseased-fungal potato,0.585861,ok,
68,p4_69.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Damaged potato,0.579927,ok,
69,p4_70.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Unable to classify,0.000000,ok,
70,p4_71.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Damaged potato,0.784698,ok,


In [18]:
import re
import pandas as pd
import numpy as np

def add_and_sort_p4(df: pd.DataFrame) -> pd.DataFrame:
    def _idx(name):
        m = re.search(r"\bp4_(\d+)\b", str(name))
        return int(m.group(1)) if m else 10**9
    out = df.copy()
    out["p4_idx"] = out["image"].apply(_idx)
    out = out.sort_values("p4_idx").reset_index(drop=True)
    return out

def print_ordered(df: pd.DataFrame, title: str):
    df = add_and_sort_p4(df)
    print(f"\n=== {title} (orden p4_1, p4_2, ...) ===")
    for _, r in df.iterrows():
        idx = int(r["p4_idx"])
        img = str(r["image"])
        cls = str(r.get("pred_class", ""))
        conf = r.get("confidence", None)
        status = str(r.get("status", ""))
        extra = []
        if conf is not None and str(conf) != "" and not pd.isna(conf):
            extra.append(f"conf={float(conf):.3f}")
        if status:
            extra.append(f"status={status}")
        print(f"p4_{idx:02d} | {img:25s} -> {cls:10s}" + (f" | {', '.join(extra)}" if extra else ""))
    display(df)

# Ordena + imprimeix
df_p4_sigmoid_normal = add_and_sort_p4(df_p4_sigmoid_normal)
df_p4_top_combo      = add_and_sort_p4(df_p4_top_combo)

print_ordered(df_p4_sigmoid_normal, "Sigmoid normal")
print_ordered(df_p4_top_combo, "TOP combinació")



=== Sigmoid normal (orden p4_1, p4_2, ...) ===
p4_01 | p4_1.png                  -> Diseased-fungal potato | conf=0.802, status=ok
p4_02 | p4_2.png                  -> Diseased-fungal potato | conf=0.847, status=ok
p4_03 | p4_3.png                  -> Diseased-fungal potato | conf=0.847, status=ok
p4_04 | p4_4.png                  -> Diseased-fungal potato | conf=0.753, status=ok
p4_05 | p4_5.png                  -> Diseased-fungal potato | conf=0.430, status=ok
p4_06 | p4_6.png                  -> Diseased-fungal potato | conf=0.914, status=ok
p4_07 | p4_7.png                  -> Damaged potato | conf=0.669, status=ok
p4_08 | p4_8.png                  -> Diseased-fungal potato | conf=0.859, status=ok
p4_09 | p4_9.png                  -> Diseased-fungal potato | conf=0.759, status=ok
p4_10 | p4_10.png                 -> Diseased-fungal potato | conf=0.803, status=ok
p4_11 | p4_11.png                 -> Diseased-fungal potato | conf=0.678, status=ok
p4_12 | p4_12.png                 ->

,image,lot,source,order,brightness,gamma,sigmoid_k,sigmoid_mid,thr,pred_class,confidence,status,error,p4_idx
0,p4_1.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Diseased-fungal potato,0.802043,ok,,1
1,p4_2.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Diseased-fungal potato,0.847370,ok,,2
2,p4_3.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Diseased-fungal potato,0.847291,ok,,3
3,p4_4.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Diseased-fungal potato,0.752796,ok,,4
4,p4_5.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Diseased-fungal potato,0.430456,ok,,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,p4_68.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Diseased-fungal potato,0.807448,ok,,68
68,p4_69.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Damaged potato,0.546293,ok,,69
69,p4_70.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Damaged potato,0.411721,ok,,70
70,p4_71.png,4,definitive,sigmoid_only,1.0,1.0,6.0,0.5,0.2,Damaged potato,0.746927,ok,,71



=== TOP combinació (orden p4_1, p4_2, ...) ===
p4_01 | p4_1.png                  -> Diseased-fungal potato | conf=0.832, status=ok
p4_02 | p4_2.png                  -> Diseased-fungal potato | conf=0.829, status=ok
p4_03 | p4_3.png                  -> Diseased-fungal potato | conf=0.839, status=ok
p4_04 | p4_4.png                  -> Diseased-fungal potato | conf=0.676, status=ok
p4_05 | p4_5.png                  -> Diseased-fungal potato | conf=0.464, status=ok
p4_06 | p4_6.png                  -> Diseased-fungal potato | conf=0.907, status=ok
p4_07 | p4_7.png                  -> Damaged potato | conf=0.639, status=ok
p4_08 | p4_8.png                  -> Diseased-fungal potato | conf=0.839, status=ok
p4_09 | p4_9.png                  -> Diseased-fungal potato | conf=0.719, status=ok
p4_10 | p4_10.png                 -> Diseased-fungal potato | conf=0.754, status=ok
p4_11 | p4_11.png                 -> Diseased-fungal potato | conf=0.570, status=ok
p4_12 | p4_12.png                 ->

,image,lot,source,order,brightness,gamma,sigmoid_k,sigmoid_mid,thr,pred_class,confidence,status,error,p4_idx
0,p4_1.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Diseased-fungal potato,0.832088,ok,,1
1,p4_2.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Diseased-fungal potato,0.829076,ok,,2
2,p4_3.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Diseased-fungal potato,0.839306,ok,,3
3,p4_4.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Diseased-fungal potato,0.675565,ok,,4
4,p4_5.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Diseased-fungal potato,0.463931,ok,,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,p4_68.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Diseased-fungal potato,0.585861,ok,,68
68,p4_69.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Damaged potato,0.579927,ok,,69
69,p4_70.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Unable to classify,0.000000,ok,,70
70,p4_71.png,4,definitive,bg_then_sigmoid,1.0,1.1,6.0,0.5,0.2,Damaged potato,0.784698,ok,,71
